In [1]:
"""Target Encoder"""
import numpy as np
import pandas as pd
from scipy.special import expit
from category_encoders.ordinal import OrdinalEncoder
import category_encoders.utils as util

In [2]:
__author__ = 'chappers'
class TargetEncoder(util.BaseEncoder, util.SupervisedTransformerMixin):
    """Target encoding for categorical features.

    Supported targets: binomial and continuous. For polynomial target support, see PolynomialWrapper.

    For the case of categorical target: features are replaced with a blend of posterior probability of the target
    given particular categorical value and the prior probability of the target over all the training data.

    For the case of continuous target: features are replaced with a blend of the expected value of the target
    given particular categorical value and the expected value of the target over all the training data.

    Parameters
    ----------

    verbose: int
        integer indicating verbosity of the output. 0 for none.
    cols: list
        a list of columns to encode, if None, all string columns will be encoded.
    drop_invariant: bool
        boolean for whether or not to drop columns with 0 variance.
    return_df: bool
        boolean for whether to return a pandas DataFrame from transform (otherwise it will be a numpy array).
    handle_missing: str
        options are 'error', 'return_nan'  and 'value', defaults to 'value', which returns the target mean.
    handle_unknown: str
        options are 'error', 'return_nan' and 'value', defaults to 'value', which returns the target mean.
    min_samples_leaf: int
        For regularization the weighted average between category mean and global mean is taken. The weight is
        an S-shaped curve between 0 and 1 with the number of samples for a category on the x-axis.
        The curve reaches 0.5 at min_samples_leaf. (parameter k in the original paper)
    smoothing: float
        smoothing effect to balance categorical average vs prior. Higher value means stronger regularization.
        The value must be strictly bigger than 0. Higher values mean a flatter S-curve (see min_samples_leaf).
    hierarchy: dict or dataframe
        A dictionary or a dataframe to define the hierarchy for mapping.

        If a dictionary, this contains a dict of columns to map into hierarchies.  Dictionary key(s) should be the column name from X
        which requires mapping.  For multiple hierarchical maps, this should be a dictionary of dictionaries.

        If dataframe: a dataframe defining columns to be used for the hierarchies.  Column names must take the form:
            HIER_colA_1, ... HIER_colA_N, HIER_colB_1, ... HIER_colB_M, ...
        where [colA, colB, ...] are given columns in cols list.  
        1:N and 1:M define the hierarchy for each column where 1 is the highest hierarchy (top of the tree).  A single column or multiple 
        can be used, as relevant.

    Examples
    -------
    >>> from category_encoders import *
    >>> import pandas as pd
    >>> from sklearn.datasets import fetch_openml
    >>> display_cols = ["Id", "MSSubClass", "MSZoning", "LotFrontage", "YearBuilt", "Heating", "CentralAir"]
    >>> bunch = fetch_openml(name="house_prices", as_frame=True)
    >>> y = bunch.target > 200000
    >>> X = pd.DataFrame(bunch.data, columns=bunch.feature_names)[display_cols]
    >>> enc = TargetEncoder(cols=['CentralAir', 'Heating'], min_samples_leaf=20, smoothing=10).fit(X, y)
    >>> numeric_dataset = enc.transform(X)
    >>> print(numeric_dataset.info())
    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 1460 entries, 0 to 1459
    Data columns (total 7 columns):
     #   Column       Non-Null Count  Dtype  
    ---  ------       --------------  -----  
     0   Id           1460 non-null   float64
     1   MSSubClass   1460 non-null   float64
     2   MSZoning     1460 non-null   object 
     3   LotFrontage  1201 non-null   float64
     4   YearBuilt    1460 non-null   float64
     5   Heating      1460 non-null   float64
     6   CentralAir   1460 non-null   float64
    dtypes: float64(6), object(1)
    memory usage: 80.0+ KB
    None
   
    >>> from category_encoders.datasets import load_compass
    >>> X, y = load_compass()
    >>> hierarchical_map = {'compass': {'N': ('N', 'NE'), 'S': ('S', 'SE'), 'W': 'W'}}
    >>> enc = TargetEncoder(verbose=1, smoothing=2, min_samples_leaf=2, hierarchy=hierarchical_map, cols=['compass']).fit(X.loc[:,['compass']], y)
    >>> hierarchy_dataset = enc.transform(X.loc[:,['compass']])
    >>> print(hierarchy_dataset['compass'].values)
    [0.62263617 0.62263617 0.90382995 0.90382995 0.90382995 0.17660024
     0.17660024 0.46051953 0.46051953 0.46051953 0.46051953 0.40332791
     0.40332791 0.40332791 0.40332791 0.40332791]
    >>> X, y = load_postcodes('binary')
    >>> cols = ['postcode']
    >>> HIER_cols = ['HIER_postcode_1','HIER_postcode_2','HIER_postcode_3','HIER_postcode_4']
    >>> enc = TargetEncoder(verbose=1, smoothing=2, min_samples_leaf=2, hierarchy=X[HIER_cols], cols=['postcode']).fit(X['postcode'], y)
    >>> hierarchy_dataset = enc.transform(X['postcode'])
    >>> print(hierarchy_dataset.loc[0:10, 'postcode'].values)
    [0.75063473 0.90208756 0.88328833 0.77041254 0.68891504 0.85012847
    0.76772574 0.88742357 0.7933824  0.63776756 0.9019973 ]

    References
    ----------

    .. [1] A Preprocessing Scheme for High-Cardinality Categorical Attributes in Classification and Prediction Problems, from
    https://dl.acm.org/citation.cfm?id=507538

    """
    prefit_ordinal = True
    encoding_relation = util.EncodingRelation.ONE_TO_ONE

    def __init__(self, verbose=0, cols=None, drop_invariant=False, return_df=True, handle_missing='value',
                 handle_unknown='value', min_samples_leaf=20, smoothing=10, hierarchy=None):
        super().__init__(verbose=verbose, cols=cols, drop_invariant=drop_invariant, return_df=return_df,
                         handle_unknown=handle_unknown, handle_missing=handle_missing)
        self.ordinal_encoder = None
        self.min_samples_leaf = min_samples_leaf
        self.smoothing = smoothing
        self.mapping = None
        self._mean = None
        if isinstance(hierarchy, (dict, pd.DataFrame)) and cols is None:
            raise ValueError('Hierarchy is defined but no columns are named for encoding')
        if isinstance(hierarchy, dict):
            self.hierarchy = {}
            self.hierarchy_depth = {}
            for switch in hierarchy:
                flattened_hierarchy = util.flatten_reverse_dict(hierarchy[switch])
                hierarchy_check = self._check_dict_key_tuples(flattened_hierarchy)
                self.hierarchy_depth[switch] = hierarchy_check[1]
                if not hierarchy_check[0]:
                    raise ValueError('Hierarchy mapping contains different levels for key "' + switch + '"')
                self.hierarchy[switch] = {(k if isinstance(t, tuple) else t): v for t, v in flattened_hierarchy.items() for k in t}
        elif isinstance(hierarchy, pd.DataFrame):
            self.hierarchy = hierarchy
            self.hierarchy_depth = {}
            for col in self.cols:
                HIER_cols = self.hierarchy.columns[self.hierarchy.columns.str.startswith(f'HIER_{col}')].tolist()
                HIER_levels = [int(i.replace(f'HIER_{col}_', '')) for i in HIER_cols]
                if np.array_equal(sorted(HIER_levels), np.arange(1, max(HIER_levels)+1)):
                    self.hierarchy_depth[col] = max(HIER_levels)
                else:
                    raise ValueError(f'Hierarchy columns are not complete for column {col}')
        elif hierarchy is None:
            self.hierarchy = hierarchy
        else:
            raise ValueError('Given hierarchy mapping is neither a dictionary nor a dataframe')

        self.cols_hier = []

    def _check_dict_key_tuples(self, d):
        min_tuple_size = min(len(v) for v in d.values())
        max_tuple_size = max(len(v) for v in d.values())
        return min_tuple_size == max_tuple_size, min_tuple_size

    def _fit(self, X, y, **kwargs):
        if isinstance(self.hierarchy, dict):
            X_hier = pd.DataFrame()
            for switch in self.hierarchy:
                if switch in self.cols:
                    colnames = [f'HIER_{str(switch)}_{str(i + 1)}' for i in range(self.hierarchy_depth[switch])]
                    df = pd.DataFrame(X[str(switch)].map(self.hierarchy[str(switch)]).tolist(), index=X.index, columns=colnames)
                    X_hier = pd.concat([X_hier, df], axis=1)
        elif isinstance(self.hierarchy, pd.DataFrame):
            X_hier = self.hierarchy

        if isinstance(self.hierarchy, (dict, pd.DataFrame)):
            enc_hier = OrdinalEncoder(
                verbose=self.verbose,
                cols=X_hier.columns,
                handle_unknown='value',
                handle_missing='value'
            )
            enc_hier = enc_hier.fit(X_hier)
            X_hier_ordinal = enc_hier.transform(X_hier)

        self.ordinal_encoder = OrdinalEncoder(
            verbose=self.verbose,
            cols=self.cols,
            handle_unknown='value',
            handle_missing='value'
        )
        self.ordinal_encoder = self.ordinal_encoder.fit(X)
        X_ordinal = self.ordinal_encoder.transform(X)
        if self.hierarchy is not None:
            self.mapping = self.fit_target_encoding(pd.concat([X_ordinal, X_hier_ordinal], axis=1), y)
        else:
            self.mapping = self.fit_target_encoding(X_ordinal, y)

    def fit_target_encoding(self, X, y):
        mapping = {}
        prior = self._mean = y.mean()

        for switch in self.ordinal_encoder.category_mapping:
            col = switch.get('col')
            if 'HIER_' not in str(col):
                values = switch.get('mapping')

                scalar = prior
                if (isinstance(self.hierarchy, dict) and col in self.hierarchy) or \
                                    (isinstance(self.hierarchy, pd.DataFrame)):
                    for i in range(self.hierarchy_depth[col]):
                        col_hier = 'HIER_'+str(col)+'_'+str(i+1)
                        col_hier_m1 = col if i == self.hierarchy_depth[col]-1 else 'HIER_'+str(col)+'_'+str(i+2)
                        if not X[col].equals(X[col_hier]) and len(X[col_hier].unique())>1:
                            stats_hier = y.groupby(X[col_hier]).agg(['count', 'mean'])
                            smoove_hier = self._weighting(stats_hier['count'])
                            scalar_hier = scalar * (1 - smoove_hier) + stats_hier['mean'] * smoove_hier
                            scalar_hier_long = X[[col_hier_m1, col_hier]].drop_duplicates()
                            scalar_hier_long.index = np.arange(1, scalar_hier_long.shape[0]+1)
                            scalar = scalar_hier_long[col_hier].map(scalar_hier.to_dict())

                stats = y.groupby(X[col]).agg(['count', 'mean'])
                smoove = self._weighting(stats['count'])

                smoothing = scalar * (1 - smoove) + stats['mean'] * smoove

                if self.handle_unknown == 'return_nan':
                    smoothing.loc[-1] = np.nan
                elif self.handle_unknown == 'value':
                    smoothing.loc[-1] = prior

                if self.handle_missing == 'return_nan':
                    smoothing.loc[values.loc[np.nan]] = np.nan
                elif self.handle_missing == 'value':
                    smoothing.loc[-2] = prior

                mapping[col] = smoothing

        return mapping

    def _transform(self, X, y=None):
        # Now X is the correct dimensions it works with pre fitted ordinal encoder
        X = self.ordinal_encoder.transform(X)

        if self.handle_unknown == 'error':
            if X[self.cols].isin([-1]).any().any():
                raise ValueError('Unexpected categories found in dataframe')

        X = self.target_encode(X)
        return X

    def target_encode(self, X_in):
        X = X_in.copy(deep=True)

        # Was not mapping extra columns as self.featuer_names_in did not include new column
        for col in self.cols:
            X[col] = X[col].map(self.mapping[col])

        return X

    def _weighting(self, n):
        # monotonically increasing function of n bounded between 0 and 1
        # sigmoid in this case, using scipy.expit for numerical stability
        return expit((n - self.min_samples_leaf) / self.smoothing)

In [3]:
from category_encoders import *
import pandas as pd
from sklearn.datasets import fetch_openml
display_cols = ["Id", "MSSubClass", "MSZoning", "LotFrontage", "YearBuilt", "Heating", "CentralAir"]
bunch = fetch_openml(name="house_prices", as_frame=True)
y = bunch.target > 200000
X = pd.DataFrame(bunch.data, columns=bunch.feature_names)[display_cols]
display(X)

,Id,MSSubClass,MSZoning,LotFrontage,YearBuilt,Heating,CentralAir
0,1,60,RL,65.0,2003,GasA,Y
1,2,20,RL,80.0,1976,GasA,Y
2,3,60,RL,68.0,2001,GasA,Y
3,4,70,RL,60.0,1915,GasA,Y
4,5,60,RL,84.0,2000,GasA,Y
...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,1999,GasA,Y
1456,1457,20,RL,85.0,1978,GasA,Y
1457,1458,70,RL,66.0,1941,GasA,Y
1458,1459,20,RL,68.0,1950,GasA,Y


In [4]:
enc = TargetEncoder(cols=['CentralAir', 'Heating']).fit(X, y)
numeric_dataset = enc.transform(X)
display(numeric_dataset)

,Id,MSSubClass,MSZoning,LotFrontage,YearBuilt,Heating,CentralAir
0,1,60,RL,65.0,2003,0.294818,0.30989
1,2,20,RL,80.0,1976,0.294818,0.30989
2,3,60,RL,68.0,2001,0.294818,0.30989
3,4,70,RL,60.0,1915,0.294818,0.30989
4,5,60,RL,84.0,2000,0.294818,0.30989
...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,1999,0.294818,0.30989
1456,1457,20,RL,85.0,1978,0.294818,0.30989
1457,1458,70,RL,66.0,1941,0.294818,0.30989
1458,1459,20,RL,68.0,1950,0.294818,0.30989


In [5]:
from category_encoders.datasets import load_compass
X, y = load_compass()
display(X)
hierarchical_map = {'compass': {'N': ('N', 'NE'), 'S': ('S', 'SE'), 'W': 'W'}}
enc = TargetEncoder(verbose=1, smoothing=2, min_samples_leaf=2, hierarchy=hierarchical_map, cols=['compass']).fit(X.loc[:,['compass']], y)
hierarchy_dataset = enc.transform(X.loc[:,['compass']])
display(hierarchy_dataset)

,index,compass,HIER_compass_1
0,1,N,N
1,2,N,N
2,3,NE,N
3,4,NE,N
4,5,NE,N
5,6,SE,S
6,7,SE,S
7,8,S,S
8,9,S,S
9,10,S,S


,compass
0,0.622636
1,0.622636
2,0.903830
3,0.903830
4,0.903830
5,0.176600
6,0.176600
7,0.460520
8,0.460520
9,0.460520


In [19]:
import numpy as np
import polars as pl
import warnings
from typing import Dict, List, Union

class PLOrdinalEncoder:
    """Encodes categorical features as ordinal, in one ordered feature.

    Ordinal encoding uses a single column of integers to represent the classes. An optional mapping dict can be passed
    in; in this case, we use the knowledge that there is some true order to the classes themselves. Otherwise, the classes
    are assumed to have no true order and integers are selected at random.

    Parameters
    ----------
    verbose: int
        integer indicating verbosity of the output. 0 for none.
    cols: list
        a list of columns to encode, if None, all string columns will be encoded.
    drop_invariant: bool
        boolean for whether or not to drop columns with 0 variance.
    return_df: bool
        boolean for whether to return a Polars DataFrame from transform (otherwise it will be a numpy array).
    mapping: list of dicts
        a mapping of class to label to use for the encoding, optional.
        the dict contains the keys 'col' and 'mapping'.
        the value of 'col' should be the feature name.
        the value of 'mapping' should be a dictionary or pl.Series of 'original_label' to 'encoded_label'.
    handle_unknown: str
        options are 'error', 'return_nan' and 'value', defaults to 'value', which will impute the category -1.
    handle_missing: str
        options are 'error', 'return_nan', and 'value, default to 'value', which treat nan as a category at fit time,
        or -2 at transform time if nan is not a category during fit.
    """
    def __init__(self, verbose=0, mapping=None, cols=None, drop_invariant=False, return_df=True,
                 handle_unknown='value', handle_missing='value'):
        self.verbose = verbose
        self.mapping_supplied = mapping is not None
        if self.mapping_supplied:
            self.mapping = self._validate_supplied_mapping(mapping)
        else:
            self.mapping = None
        self.cols = cols
        self.drop_invariant = drop_invariant
        self.return_df = return_df
        self.handle_unknown = handle_unknown
        self.handle_missing = handle_missing

    @property
    def category_mapping(self):
        return self.mapping

    def _fit(self, X, y=None, **kwargs):
        try:
            _, categories = self.ordinal_encoding(
                X,
                mapping=self.mapping,
                cols=self.cols,
                handle_unknown=self.handle_unknown,
                handle_missing=self.handle_missing
            )
        except Exception as e:
            print(f"Error: {e}")
            print("Columns in DataFrame:", X.columns)
            raise
        """
        display(self.cols)
        if not self.mapping_supplied:
            self.mapping = None
        _, categories = self.ordinal_encoding(
            X,
            mapping=self.mapping,
            cols=self.cols,
            handle_unknown=self.handle_unknown,
            handle_missing=self.handle_missing
        )
        self.mapping = categories
        """
    def _transform(self, X):
        X, _ = self.ordinal_encoding(
            X,
            mapping=self.mapping,
            cols=self.cols,
            handle_unknown=self.handle_unknown,
            handle_missing=self.handle_missing
        )
        return X

    def inverse_transform(self, X: pl.DataFrame):
        if self._dim is None:
            raise ValueError('Must train encoder before it can be used to inverse_transform data')

        if X.shape[1] != self._dim:
            raise ValueError(f'Unexpected input dimension {X.shape[1]}, expected {self._dim}')

        if not self.cols:
            return X if self.return_df else X.to_numpy()

        for switch in self.mapping:
            column_mapping = switch.get('mapping')
            inverse = {v: k for k, v in column_mapping.items()}
            X = X.with_columns(pl.col(switch.get('col')).apply(lambda x: inverse.get(x, x)))

        return X if self.return_df else X.to_numpy()

    @staticmethod
    def ordinal_encoding(X_in: pl.DataFrame, mapping=None, cols=None, handle_unknown='value', handle_missing='value'):
        if cols is None:
            cols = X_in.columns
        else:
            # 確認している列がDataFrameに存在するかどうかをチェック
            missing_cols = [col for col in cols if col not in X_in.columns]
            if missing_cols:
                raise ValueError(f"Columns not found in DataFrame: {missing_cols}")
            
        if mapping is not None:
            mapping_out = mapping
            for switch in mapping:
                column = switch.get('col')
                col_mapping = switch['mapping']
                X_in = X_in.with_columns(
                    pl.when(X_in[column].is_in(list(col_mapping.keys())))
                    .then(pl.element().map_batches(col_mapping))
                    .otherwise(-1 if handle_unknown == 'value' else pl.element())
                    .alias(column)
                )
                if handle_missing == 'return_nan':
                    X_in = X_in.with_columns(
                        pl.when(X_in[column].is_null())
                        .then(-2)
                        .otherwise(X_in[column])
                        .alias(column)
                    )
        else:
            mapping_out = []
            for col in cols:
                categories = X_in[col].unique().to_numpy()
                index = {val: idx + 1 for idx, val in enumerate(categories) if val is not None}

                if handle_missing == 'value':
                    index[None] = -2  # Use None for missing values in polars

                mapping_out.append({'col': col, 'mapping': index, 'data_type': X_in[col].dtype})

                X_in = X_in.with_columns(
                    pl.when(X_in[col].is_in(list(index.keys())))
                    .then(pl.element().map_batches(index))
                    .otherwise(-1 if handle_unknown == 'value' else pl.element())
                    .alias(col)
                )

        return X_in, mapping_out


    def _validate_supplied_mapping(self, supplied_mapping: List[Dict[str, Union[str, Dict, pl.Series]]]) -> List[Dict[str, Union[str, pl.Series]]]:
        if not isinstance(supplied_mapping, list):
            raise ValueError("Invalid supplied mapping, must be of type List[Dict[str, Union[Dict, pl.Series]]].")
        for mapping_el in supplied_mapping:
            if not isinstance(mapping_el, dict):
                raise ValueError("Invalid supplied mapping, must be of type List[Dict[str, Union[Dict, pl.Series]]].")
            if "col" not in mapping_el:
                raise KeyError("Mapping must contain a key 'col' for each column to encode")
            if "mapping" not in mapping_el:
                raise KeyError("Mapping must contain a key 'mapping' for each column to encode")
            mapping = mapping_el["mapping"]
            if isinstance(mapping, dict):
                mapping_el["mapping"] = pl.Series(list(mapping.keys()), values=list(mapping.values()), dtype=pl.Categorical)
            if "data_type" not in mapping_el:
                mapping_el["data_type"] = mapping_el["mapping"].dtype
        return supplied_mapping


In [20]:
from category_encoders import *
import pandas as pd
from sklearn.datasets import fetch_openml
bunch = fetch_openml(name="house_prices", as_frame=True)
display_cols = ["Id", "MSSubClass", "MSZoning", "LotFrontage", "YearBuilt", "Heating", "CentralAir"]
y = bunch.target
X = pl.DataFrame(bunch.data)[display_cols]
display(X)
encoder = PLOrdinalEncoder(cols=['CentralAir', 'Heating'])._fit(X, y)
numeric_dataset = enc.transform(X)
#print(numeric_dataset.info())

In [29]:
def ordinal_encoding(X_in: pl.DataFrame, mapping=None, cols=None, handle_unknown='value', handle_missing='value'):
        if cols is None:
            cols = X_in.columns
        else:
            # 確認している列がDataFrameに存在するかどうかをチェック
            missing_cols = [col for col in cols if col not in X_in.columns]
            if missing_cols:
                raise ValueError(f"Columns not found in DataFrame: {missing_cols}")
            
        if mapping is not None:
            mapping_out = mapping
            for switch in mapping:
                column = switch.get('col')
                col_mapping = switch['mapping']
                X_in = X_in.with_columns(
                    pl.when(X_in[column].is_in(list(col_mapping.keys())))
                    .then(pl.element().map_batches(col_mapping))
                    .otherwise(-1 if handle_unknown == 'value' else pl.element())
                    .alias(column)
                )
                if handle_missing == 'return_nan':
                    X_in = X_in.with_columns(
                        pl.when(X_in[column].is_null())
                        .then(-2)
                        .otherwise(X_in[column])
                        .alias(column)
                    )
        else:
            mapping_out = []
            for col in cols:
                categories = X_in[col].unique().to_numpy()
                index = {val: idx + 1 for idx, val in enumerate(categories) if val is not None}

                if handle_missing == 'value':
                    index[None] = -2  # Use None for missing values in polars

                mapping_out.append({'col': col, 'mapping': index, 'data_type': X_in[col].dtype})

                X_in = X_in.with_columns(
                    pl.when(X_in[col].is_in(list(index.keys())))
                    .then(pl.element().map_batches(index))
                    .otherwise(-1 if handle_unknown == 'value' else pl.element())
                    .alias(col)
                )

        return X_in, mapping_out
    
def fit(X_in: pl.DataFrame, mapping=None, cols=None, handle_unknown='value', handle_missing='value'):
    try:
        _, categories = ordinal_encoding(
            X,
            mapping=mapping,
            cols=cols,
            handle_unknown=handle_unknown,
            handle_missing=handle_missing
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Columns in DataFrame:", X.columns)
        raise

Id,MSSubClass,MSZoning,LotFrontage,YearBuilt,Heating,CentralAir
i64,i64,str,f64,i64,str,str
1,60,"""RL""",65.0,2003,"""GasA""","""Y"""
2,20,"""RL""",80.0,1976,"""GasA""","""Y"""
3,60,"""RL""",68.0,2001,"""GasA""","""Y"""
4,70,"""RL""",60.0,1915,"""GasA""","""Y"""
5,60,"""RL""",84.0,2000,"""GasA""","""Y"""
…,…,…,…,…,…,…
1456,60,"""RL""",62.0,1999,"""GasA""","""Y"""
1457,20,"""RL""",85.0,1978,"""GasA""","""Y"""
1458,70,"""RL""",66.0,1941,"""GasA""","""Y"""


Error: 
Columns in DataFrame: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'YearBuilt', 'Heating', 'CentralAir']


ColumnNotFoundError: 